<a href="https://colab.research.google.com/github/JasonKeha/Scraping_Projects/blob/main/Amsterdam_Housing_(Scraping_Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python data scraping project

Needed libraries are below

In [ ]:
import requests
from bs4 import BeautifulSoup
from csv import writer
import os.path
from os import path 
from statistics import mean
import sys
from math import ceil
import time

Checking if the final file already exists

In [ ]:
if path.exists('Amsterdam_Housing.csv'):
    print('File already exists!!')  
    while True:
        delete = input('Delete file to proceed?(Yes, No) : ').lower().strip()
        if delete != 'yes' and delete != 'no':
            print('\nPlease enter Yes or No!!\n')
            continue
        if delete == 'yes':
            os.remove('Amsterdam_Housing.csv')
            break
        else:
            print('\nCant run because file already exists...')
            sys.exit()

Function for extracting the total listings and the website content
- First time this function is running, it is going to return the total rent listings 
- Second time and afterwards, it is going to return the total content of the page
- After the second time running, the change in the page variable is going to lead to the url changing to the second, (third, fourth, etc) webpage of rent listings

In [ ]:
def extract(page, first_time):
    url = f'https://www.pararius.com/apartments/amsterdam/page-{page}'
    www = requests.get(url)
    soup = BeautifulSoup(www.content, 'html.parser')
    if first_time is True:
        total_listings = int(soup.find('span', class_="search-list-header__count").getText())
        return total_listings
    else:
        return soup

Function for copying the correct webpage details and appending them to a csv file
* I take all the variables then convert the numeric ones to integers  
* I then "paste" them to a newly created csv file
* The first time this function runs I also create headers for that csv file   



In [ ]:
def transform(soup):
    lists = soup.find_all('section', class_='listing-search-item')
    print(lists)
    for list in lists:
      if posting <= total_postings:
        name = list.find('a', class_='listing-search-item__link listing-search-item__link--title').getText().strip()
        location = list.find('div', class_='listing-search-item__sub-title').getText().strip()
        price = list.find('div', class_='listing-search-item__price').text
        area = list.find('li', class_='illustrated-features__item illustrated-features__item--surface-area').getText().strip()
        rooms = list.find('li', class_='illustrated-features__item illustrated-features__item--number-of-rooms').getText().strip()
        company = list.find('div', class_='listing-search-item__info').getText().strip() 
        try:
          furniture = list.find('li', class_='illustrated-features__item illustrated-features__item--interior').getText().strip()
        except AttributeError:
          furniture = 'Not Provided'
        try:
          age = list.find('li', class_='illustrated-features__item illustrated-features__item--construction-period').getText().strip()
        except AttributeError:  
          age = 'Not Provided'
        area = int(area[0:3])
        rooms = int(''.join([number for number in rooms if number.isdigit()]))
        if '-' in price: # If the price is represented in a range (€1550 - €1650) then I take the average price between those ranges.
          price = ''.join([numsymb for numsymb in price if numsymb.isalpha() == False and numsymb != '€' and numsymb != ',' and numsymb != ' '])
          price_list = price.split('-')
          price = [int(i) for i in price_list]
          price = round(mean(price), 0)
          price_per_m = int(price/area)
        else: 
          price = int(''.join([number for number in price if number.isdigit()]))
          price_per_m = int(price/area)

        info = [name, location, price, area, price_per_m, rooms, furniture, age, company]
        with open('Amsterdam_Housing.csv', 'a', encoding='utf8', newline='') as f:
          thewriter = writer(f)
          if posting == 1 and page == 1:
            header = ['Name', 'Location', 'Price in Euro', 'Area m²', 'Price per m²', 'Rooms', 'Furniture', 'Age', 'Company']
            thewriter.writerow(header)
          thewriter.writerow(info)
        posting += 1

Assigning the page, rent posting and finding the total rent listings 

In [ ]:
page = 1
posting = 1
total_listings = extract(page, True)

Asking the user the total rent listings he wants the program to take from the webpage

In [ ]:
total_listings = 920

In [ ]:
while True:
    total_postings =input(f'\nInput the total number of listings you want me to look at.\nMinimum Number (32)\nRecommended Number ({total_listings}): ')
    if total_postings.isdigit():
        total_postings = int(total_postings)
        print(total_postings)
        total_pages = ceil(total_postings/31)
        print(f'Processing {total_pages} pages\n')
        break
    else:
        print('Please enter a number next time!! \n')

Place that all the functions run

In [ ]:
while page <= total_pages:
    soup = extract(page, False)
    transform(soup)
    page += 1

Final Message to the user

In [ ]:
print('Check the newly attached csv file for the results.\nTo properly open a csv file you can check: https://www.softwaretestinghelp.com/how-to-open-csv-file/#:~:text=Answer%3A%20You%20can%20open%20the,want%20to%20open%2C%20click%20import.')